In [33]:
%pip install pysoundfile
%pip install pedalboard
%pip install torchaudio
%pip install scipy

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p36/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [37]:
import matplotlib.pyplot as plt
import numpy as np
import wave
import sys
import math
import contextlib
import torch
import torchaudio
import math
from IPython.display import Audio
from pedalboard import (
    Pedalboard,
    Convolution,
    Distortion,
    Limiter
)

In [ ]:
file_name = "data_segments/0_1.wav"
out_name = "Test/noise3.wav"

In [14]:
def LowPassFilter(fname, outname, intensity):
    cutOffFrequency = math.exp(intensity) * (-11057.55743) + 31057.55743 
    print(cutOffFrequency)
    with contextlib.closing(wave.open(fname,'rb')) as spf:
        sampleRate = spf.getframerate()
        ampWidth = spf.getsampwidth()
        nChannels = spf.getnchannels()
        nFrames = spf.getnframes()

        # Extract Raw Audio from multi-channel Wav File
        signal = spf.readframes(nFrames*nChannels)
        spf.close()
        channels = interpret_wav(signal, nFrames, nChannels, ampWidth, True)

        # get window size
        # from http://dsp.stackexchange.com/questions/9966/what-is-the-cut-off-frequency-of-a-moving-average-filter
        freqRatio = (cutOffFrequency/sampleRate)
        N = int(math.sqrt(0.196196 + freqRatio**2)/freqRatio)

        # Use moviung average (only on first channel)
        filtered = running_mean(channels[0], N).astype(channels.dtype)

        wav_file = wave.open(outname, "w")
        wav_file.setparams((1, ampWidth, sampleRate, nFrames, spf.getcomptype(), spf.getcompname()))
        wav_file.writeframes(filtered.tobytes('C'))
        wav_file.close()

    
    
def running_mean(x, windowSize):
    cumsum = np.cumsum(np.insert(x, 0, 0)) 
    return (cumsum[windowSize:] - cumsum[:-windowSize]) / windowSize

def interpret_wav(raw_bytes, n_frames, n_channels, sample_width, interleaved = True):

    if sample_width == 1:
        dtype = np.uint8 # unsigned char
    elif sample_width == 2:
        dtype = np.int16 # signed 2-byte short
    elif sample_width == 4:
        dtype = np.int32
    else:
        raise ValueError("Only supports 8 and 16 bit audio formats.")

    channels = np.fromstring(raw_bytes, dtype=dtype)

    if interleaved:
        # channels are interleaved, i.e. sample N of channel M follows sample N of channel M-1 in raw data
        channels.shape = (n_frames, n_channels)
        channels = channels.T
    else:
        # channels are not interleaved. All samples from channel M occur before all samples from channel M-1
        channels.shape = (n_channels, n_frames)

    return channels

In [29]:
def Noise(fname, outname, intensity):
    c = 0.0999*intensity + 0.0001
    signal, sr = torchaudio.load(fname)
    signal = signal.numpy()
    noise = np.random.normal(0, .1, signal.shape)
    signal_noise = signal+c*noise
    a = Audio(signal_noise, rate=sr)
    with open(outname, 'wb') as f:
        f.write(a.data)

1000.0000008887


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel/__main__.py:45: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


In [30]:
def Distort(fname, outname, intensity):
    sig, sr = torchaudio.load(fname)
    board = Pedalboard([
    Distortion(drive_db = 25*intensity)
    ], sample_rate=sr)
    effected = board(sig)
    a = Audio(effected, rate=sr)
    with open(outname, 'wb') as f:
        f.write(a.data)

In [54]:
def Limit(fname, outname, intensity):
    sig, sr = torchaudio.load(fname)
    board = Pedalboard([
    Limiter(threshold_db = -30*intensity, release_ms = 10)
    ], sample_rate=sr)
    effected = board(sig)
    a = Audio(effected, rate=sr)
    with open(outname, 'wb') as f:
        f.write(a.data)

In [55]:
file_name = "data_segments/0_1.wav"
out_name = "Test/noise3.wav"
Limit(file_name, out_name, 0.3)